In [2]:
import torch
import numpy as np

In [8]:
batch_size = 5
truncated_length = 100

lengths = torch.tensor([800,732,658,147,329,76,62,261])
batch = []
len = 0
processed = 0
batch_lengths = lengths[:batch_size] # starting batch
while True:
    batch_lengths = torch.cat((batch_lengths, lengths[processed+batch_size:processed+batch_size+(batch_size - batch_lengths.shape[0])]), 0)
    print("batch_lengths =", batch_lengths, "self.len =", len, "processed =",processed)
    if (batch_lengths.shape[0] < batch_size):
        break
    batch_lengths = (batch_lengths-truncated_length)[batch_lengths-truncated_length>0] # theoretical length of the elements in the next batch that came from this batch.
    processed+=batch_size-batch_lengths.shape[0]
    len += 1
print("len =",len)

batch_lengths = tensor([800, 732, 658, 147, 329]) self.len = 0 processed = 0
batch_lengths = tensor([700, 632, 558,  47, 229]) self.len = 1 processed = 0
batch_lengths = tensor([600, 532, 458, 129,  62]) self.len = 2 processed = 1
batch_lengths = tensor([500, 432, 358,  29, 261]) self.len = 3 processed = 2
batch_lengths = tensor([400, 332, 258, 161]) self.len = 4 processed = 3
len = 4


# Produce index array containing filelist index and start frame

In [30]:
batch_size = 5
truncated_length = 100

dataloader_indexes = []
"""[
    [0, 0], # [index, index_frame_offset]
    [1, 0],
    [2, 0],
    [3, 0],
    [4, 0], # end of iter 1
    [5, 0], # start of iter 2
    [1, 100],
    [2, 100],
    [3, 100],
    [4, 100], # end of iter 2
]"""

lengths = torch.tensor([80,732,358,147,329,76,62,261])
batch = []
len = 0
processed = 0
batch_lengths = lengths[:batch_size] # starting batch
batch_indexes = torch.arange(batch_size)
while True:
    print("processed =", processed, "batch_indexes =", batch_indexes, "batch_lengths =", batch_lengths)
    batch_lengths = batch_lengths-truncated_length
    processed+=(batch_lengths<1).sum().item()
    batch_indexes[batch_lengths<1] = lengths[processed+batch_size:processed+batch_size]
    len += 1
    batch_lengths[batch_lengths<1] = lengths[processed+batch_size:processed+batch_size+(batch_lengths<1).sum().item()]
    print("----------------------")

    if len > 20: break
print("len =",len)

processed = 0 batch_indexes = tensor([0, 1, 2, 3, 4]) batch_lengths = tensor([ 80, 732, 358, 147, 329])
----------------------
processed = 1 batch_indexes = tensor([0, 1, 2, 3, 4]) batch_lengths = tensor([ 62, 632, 258,  47, 229])


RuntimeError: shape mismatch: value tensor of shape [0] cannot be broadcast to indexing result of shape [2]

In [ ]:

# [80 , 160, 240, 320, 400] # [ 0, 1, 2, 3, 4] # start
# [80 , 160, 240, 320, 400] # [ 0, 1, 2, 3, 4] # start
# [-20,  60, 140, 220, 300] # [ 0, 1, 2, 3, 4] # truncate

# [400,  60, 140, 220, 300] # [ 5, 1, 2, 3, 4] # replace
# [400,  60, 140, 220, 300] # [ 5, 1, 2, 3, 4] # start
# [300, -40,  40, 120, 200] # [ 5, 1, 2, 3, 4] # truncate

# [300, 480,  40, 120, 200] # [ 5, 6, 2, 3, 4] # replace
# [300, 480,  40, 120, 200] # [ 5, 6, 2, 3, 4] # start
# [200, 380, -60,  20, 100] # [ 5, 6, 2, 3, 4] # truncate

In [84]:
batch_size = 6
n_gpus = 1
total_batch_size = batch_size * n_gpus # number of audio files being processed simultainously
truncated_lengths = 100 # frames

dataloader_indexes = []
"""[
    [0, 0], # [index, index_frame_offset]
    [1, 0],
    [2, 0],
    [3, 0],
    [4, 0], # end of iter 1
    [5, 0], # start of iter 2
    [1, 100],
    [2, 100],
    [3, 100],
    [4, 100], # end of iter 2
]"""

#audio_lengths = (torch.arange(100)+1)*80
audio_lengths = torch.randint(80,800,(15,))
print(audio_lengths)

batch_remaining_lengths = audio_lengths[:total_batch_size]
batch_frame_offset = torch.zeros(total_batch_size)
batch_indexes = torch.tensor(list(range(total_batch_size)))
processed = 0

print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

while audio_lengths.shape[0]+1>processed+total_batch_size+currently_empty_lengths:
    # replace empty lengths
    currently_empty_lengths = (batch_remaining_lengths<1).sum().item()
    # update batch_indexes
    batch_indexes[batch_remaining_lengths<1] = torch.arange(processed+total_batch_size, processed+total_batch_size+currently_empty_lengths)
    # update batch_frame_offset
    batch_frame_offset[batch_remaining_lengths<1] = 0
    # update batch_remaining_lengths
    batch_remaining_lengths[batch_remaining_lengths<1] = audio_lengths[processed+total_batch_size:processed+total_batch_size+currently_empty_lengths]
    
    # update how many audiofiles have been fully used
    processed+=currently_empty_lengths
    
    dataloader_indexes.append(list(zip(batch_indexes.numpy(), batch_frame_offset.numpy())))
    print(batch_remaining_lengths, batch_indexes, sep="\n")
    
    batch_remaining_lengths = batch_remaining_lengths - truncated_lengths # truncate batch
    batch_frame_offset = batch_frame_offset + truncated_lengths
    print(batch_remaining_lengths, "---------------------", sep="\n")

print("\n".join([str(x) for x in dataloader_indexes]))

tensor([274, 453, 186, 498, 202, 283,  94, 697,  98, 373, 673, 561, 363, 154,
        190])
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
tensor([274, 453, 186, 498, 202, 283])
tensor([0, 1, 2, 3, 4, 5])
tensor([174, 353,  86, 398, 102, 183])
---------------------
tensor([174, 353,  86, 398, 102, 183])
tensor([0, 1, 2, 3, 4, 5])
tensor([ 74, 253, -14, 298,   2,  83])
---------------------
tensor([ 74, 253,  94, 298,   2,  83])
tensor([0, 1, 6, 3, 4, 5])
tensor([-26, 153,  -6, 198, -98, -17])
---------------------
tensor([697, 153,  98, 198, 373, 673])
tensor([ 7,  1,  8,  3,  9, 10])
tensor([597,  53,  -2,  98, 273, 573])
---------------------
tensor([597,  53, 561,  98, 273, 573])
tensor([ 7,  1, 11,  3,  9, 10])
tensor([497, -47, 461,  -2, 173, 473])
---------------------
tensor([497, 363, 461, 154, 173, 473])
tensor([ 7, 12, 11, 13,  9, 10])
tensor([397, 263, 361,  54,  73, 373])
---------------------
[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0)]
[(0, 100.0), (1, 1

In [76]:
torch.tensor(False)

tensor(False)

In [77]:
x = torch.rand(6)
print(x)
x*=555
print(x)

tensor([0.0101, 0.2086, 0.6089, 0.9950, 0.6090, 0.9500])
tensor([  5.5931, 115.7942, 337.9148, 552.2272, 338.0084, 527.2679])


In [85]:
%%timeit
if 1:
    pass
else:
    pass
if 1:
    pass
else:
    pass
if 1:
    pass
else:
    pass
if 1:
    pass
else:
    pass
if 1:
    pass
else:
    pass
if 1:
    pass
else:
    pass
if 1:
    pass
else:
    pass
if 1:
    pass
else:
    pass
if 1:
    pass
else:
    pass
if 1:
    pass
else:
    pass
if 1:
    pass
else:
    pass

6.88 ns ± 0.0437 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)
